<a href="https://colab.research.google.com/github/Rstam59/TaskDataRepoForStudents/blob/main/Mnist_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#High level

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
loader = DataLoader(dataset, batch_size = 60000, shuffle = True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 502kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.63MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.71MB/s]


In [2]:
images, _ = next(iter(loader))

In [3]:
mean = images.mean()
std = images.std()

print(f"Calculated mean: {mean}")
print(f"Calculated std: {std}")

Calculated mean: 0.13066047430038452
Calculated std: 0.30810782313346863


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])


In [5]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = nn.Sequential(
    nn.Conv2d(1, 32, 3, 1),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(32, 64, 3, 1),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(64 * 5 * 5, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
).to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [8]:
def train_epoch(model, loader):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        y_pred = model(images)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Loss: {total_loss / len(loader)}")


def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            y_pred = model(images).argmax(dim = 1)
            total += labels.size(0)
            correct += (y_pred == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}")

In [9]:
for _ in range(3):
    train_epoch(model, train_loader)
    evaluate(model, test_loader)

Train Loss: 0.13912730698120285
Accuracy: 98.47
Train Loss: 0.04506941373931117
Accuracy: 98.51
Train Loss: 0.03109716234154312
Accuracy: 98.93


#Low level

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# === 1. Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 2. Calculate mean and std (optional - can hardcode 0.1307, 0.3081)
tmp_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
tmp_loader = DataLoader(tmp_dataset, batch_size=60000, shuffle=False)
imgs, _ = next(iter(tmp_loader))
mean = imgs.mean()
std = imgs.std()

# === 3. Define transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])

# === 4. Prepare datasets
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000)

# === 5. Define model using low-level API
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)   # [B, 1, 28, 28] -> [B, 32, 26, 26]
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)          # [B, 32, 13, 13]

        self.conv2 = nn.Conv2d(32, 64, 3, 1)  # [B, 64, 11, 11]
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)          # [B, 64, 5, 5]

        self.flatten = nn.Flatten()           # [B, 64*5*5]
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)

# === 6. Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === 7. Training loop
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train loss: {total_loss / len(loader):.4f}")

# === 8. Evaluation
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    print(f"Test accuracy: {100 * correct / total:.2f}%")

# === 9. Run training
for epoch in range(1, 6):
    print(f"Epoch {epoch}")
    train_one_epoch(model, train_loader)
    evaluate(model, test_loader)


Epoch 1
Train loss: 0.1406
Test accuracy: 98.65%
Epoch 2
Train loss: 0.0431
Test accuracy: 98.70%
Epoch 3
Train loss: 0.0306
Test accuracy: 99.10%
Epoch 4
Train loss: 0.0216
Test accuracy: 99.24%
Epoch 5
Train loss: 0.0160
Test accuracy: 98.95%
